In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
plt.rcParams['figure.figsize'] = [15, 10]

In [ ]:
r=100
N=50
K=100
stable_states = np.linspace(0, 1, N)

t0=0
tf=20

t = np.linspace(t0, tf, r)

x = np.zeros(t.size)
x[(0.2 < t) & (t < 0.3)] = 1.0
x[(2.5 < t) & (t < 3.5)] = 1.0
x[(7.5 < t) & (t < 8)] = -1.0
x[(12 < t) & (t < 15)] = (t[(12 < t) & (t < 15)]-12)*0.05
x = interp1d(t, x * 0.1)

integral = solve_ivp(lambda t, y: np.array([x(t)]), (t0, tf), np.array([0]), t_eval=t)
z = interp1d(t, integral.y[0])

def B(x, s):
    a = 4*N
    exp = np.exp(-a*(x-s))
    return exp/(1 + exp*exp)

def dydt(x, y):
    return np.sum(B(x, stable_states)*(stable_states - y))

def f(t, y):
    return K*np.array([dydt(x(t) * 0.2 + y*1.005, y)])

#x = 0.03
#y = x
#print(B(x, stable_states)**2*(stable_states - y))
#print(dydt(0.03, 0.03))

solution = solve_ivp(f, (t0, tf), np.array([0]), t_eval=t)
plt.plot(t, x(t), label="x(t)")
plt.plot(t, z(t), label="true integral")
plt.plot(solution.t, solution.y[0], label="y(t)")
plt.legend()

In [ ]:
u, v = np.meshgrid(t, np.linspace(0, 1, r))
plt.quiver(u, v, 1.0, K*np.array([[dydt(u__, v__) for u__, v__ in zip(u_, v_)] for u_, v_ in zip(v, v)]))

In [ ]:
y = np.array([B(x, stable_states) for x in np.linspace(0, 1, r)]).T
#print(y)

for ys in y:
    plt.plot(t, ys)